<a href="https://colab.research.google.com/github/OneFineStarstuff/Onefinebot/blob/main/N_Body_Simulation_of_Gravitational_Interactions_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Constants
G = 6.67430e-11  # Gravitational constant, in m^3 kg^-1 s^-2
MASS_SCALE = 1.989e30  # Scale mass to solar mass
TIME_STEP = 1e4  # Time step in seconds
NUM_BODIES = 10  # Number of bodies

# Initialize positions and velocities randomly
np.random.seed(0)
positions = np.random.rand(NUM_BODIES, 2) * 1e18  # positions in meters
velocities = np.random.randn(NUM_BODIES, 2) * 1e3  # velocities in m/s
masses = np.ones(NUM_BODIES) * MASS_SCALE  # masses in kg (1 solar mass each)

def compute_gravitational_force(positions, masses):
    forces = np.zeros_like(positions)
    for i in range(NUM_BODIES):
        for j in range(i + 1, NUM_BODIES):
            delta_pos = positions[j] - positions[i]
            distance = np.linalg.norm(delta_pos) + 1e10  # softening factor to avoid singularity
            force_magnitude = G * masses[i] * masses[j] / distance**2
            force_direction = delta_pos / distance
            force = force_magnitude * force_direction
            forces[i] += force
            forces[j] -= force  # Newton's third law
    return forces

def update_positions_and_velocities(positions, velocities, masses):
    forces = compute_gravitational_force(positions, masses)
    # Update velocities and positions using Euler's method
    velocities += forces / masses[:, None] * TIME_STEP
    positions += velocities * TIME_STEP
    return positions, velocities

# Run simulation and plot
num_steps = 500
plt.ion()
for step in range(num_steps):
    positions, velocities = update_positions_and_velocities(positions, velocities, masses)
    plt.clf()
    plt.scatter(positions[:, 0], positions[:, 1], s=10)
    plt.xlim(-1e18, 1e18)
    plt.ylim(-1e18, 1e18)
    plt.title(f"Step {step}")
    plt.pause(0.01)

plt.ioff()
plt.show()

In [ ]:
pip install cupy

In [ ]:
import cupy as cp
cp.show_config()

In [ ]:
import cupy as cp
import matplotlib.pyplot as plt

# Constants
G = 6.67430e-11  # Gravitational constant, in m^3 kg^-1 s^-2
MASS_SCALE = 1.989e30  # Scale mass to solar mass
TIME_STEP = 1e4  # Time step in seconds
NUM_BODIES = 10  # Number of bodies

# Initialize positions and velocities randomly
cp.random.seed(0)
positions = cp.random.rand(NUM_BODIES, 3) * 1e18  # positions in meters
velocities = cp.random.randn(NUM_BODIES, 3) * 1e3  # velocities in m/s
masses = cp.random.rand(NUM_BODIES) * MASS_SCALE  # random masses in kg

def compute_gravitational_force(positions, masses):
    forces = cp.zeros_like(positions)
    for i in range(NUM_BODIES):
        for j in range(i + 1, NUM_BODIES):
            delta_pos = positions[j] - positions[i]
            distance = cp.linalg.norm(delta_pos) + 1e10  # softening factor to avoid singularity
            force_magnitude = G * masses[i] * masses[j] / distance**2
            force_direction = delta_pos / distance
            force = force_magnitude * force_direction
            forces[i] += force
            forces[j] -= force  # Newton's third law
    return forces

def update_positions_and_velocities(positions, velocities, masses):
    def rk4_step(positions, velocities):
        k1_pos = velocities * TIME_STEP
        k1_vel = compute_gravitational_force(positions, masses) / masses[:, None] * TIME_STEP

        k2_pos = (velocities + k1_vel / 2) * TIME_STEP
        k2_vel = compute_gravitational_force(positions + k1_pos / 2, masses) / masses[:, None] * TIME_STEP

        k3_pos = (velocities + k2_vel / 2) * TIME_STEP
        k3_vel = compute_gravitational_force(positions + k2_pos / 2, masses) / masses[:, None] * TIME_STEP

        k4_pos = (velocities + k3_vel) * TIME_STEP
        k4_vel = compute_gravitational_force(positions + k3_pos, masses) / masses[:, None] * TIME_STEP

        new_positions = positions + (k1_pos + 2 * k2_pos + 2 * k3_pos + k4_pos) / 6
        new_velocities = velocities + (k1_vel + 2 * k2_vel + 2 * k3_vel + k4_vel) / 6
        return new_positions, new_velocities

    positions, velocities = rk4_step(positions, velocities)
    return positions, velocities

# Run simulation and plot
num_steps = 500
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
plt.ion()
for step in range(num_steps):
    positions, velocities = update_positions_and_velocities(positions, velocities, masses)
    ax.cla()
    ax.scatter(positions[:, 0].get(), positions[:, 1].get(), positions[:, 2].get(), s=10)
    ax.set_xlim([-1e18, 1e18])
    ax.set_ylim([-1e18, 1e18])
    ax.set_zlim([-1e18, 1e18])
    ax.set_title(f"Step {step}")
    plt.pause(0.01)

plt.ioff()
plt.show()